In [70]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product 
import pandas as pd
from sklearn.cluster import KMeans
from time import time
import os
%matplotlib qt

In [65]:
def run_sudoku(E,I,eps = 0.01,time = 300):
    n = 81

    space = np.zeros((n,int(time/eps)))
    
    metric = np.inf*np.ones(9)

    def U(x,gamma=1,I=1.01):
        tif = -1/gamma*np.log(1-(gamma/I))
        return I/gamma*(1-np.exp(-tif*x))

    def H(x,strength,gamma=1,I=1.01):
        tif = -1/gamma*np.log(1-(gamma/I))
        return 1/tif * np.log(1/(np.exp(-tif*x)-(gamma/I)*strength))

    inh = np.load("sudoku.npy")
    exi = 1-inh
    np.fill_diagonal(exi,0.0)
    
    state = np.random.normal(size=n)%1

    for i in range(int(time/eps)):

        fire_time = np.min(1-state)   

        state = state + fire_time

        new_state = (state+eps)

        m = 0
        while np.sum(new_state>1)>0:
            if m>1000:
                return False,state,metric
            m+=1

            fire = new_state>1

            new_state[fire] = 0.0

            coupling_exi = E*(np.matmul(exi,fire)>0)
            coupling_inh = I*np.matmul(inh,fire)

            total = coupling_exi-coupling_inh

            state = new_state
            new_state = np.clip(H(state,total),a_min=0,a_max=None)
            
            if np.isnan(new_state.sum()):
                return False,state,metric

        state = new_state
        space[:,i] = state
    
    
    df = pd.DataFrame(space[:,-100:])
    model = KMeans(n_clusters=9)
    pred = model.fit_predict(df)
    
    for i in range(9):
        temp = np.arange(0,81,1,dtype=np.int32)[pred==i]
        sum_ = 0
        for j,k in product(temp,temp):
            sum_ += inh[j,k]
        metric[i]=sum_
    
    return True,space,metric

In [66]:
E = 1/10**np.linspace(1,5,2)
I = 1/10**np.linspace(1,5,2)

In [ ]:
succ_mat = np.zeros((E.shape[0],I.shape[0]))
metric_mat = np.zeros((E.shape[0],I.shape[0]))
time_mat = np.zeros((E.shape[0],I.shape[0]))

for x,e in enumerate(E):
    for y,i in enumerate(I):
        t = time()
        success,space,metric = run_sudoku(e,i)
        time_mat[x,y] = time()-t
        succ_mat[x,y] = success
        metric_mat[x,y] = np.sum(metric)
        
        dirname ="E "+str(e)+" I "+str(i)
        os.mkdir(dirname)
        np.save(dirname+'/as',space)

/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in log
  


In [68]:
metric_mat

array([[ inf,  inf],
       [525., 183.]])